In [1]:
from requests_html import HTMLSession, HTML
import requests
import json
import time
import pandas as pd
import datetime as dt
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
import urllib.robotparser

In [2]:
# can fetch
url = 'https://tiki.vn/dien-thoai-smartphone/c1795?_lc=Vk4wMzkwMjIwMDk%3D'
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://tiki.vn/robots.txt')
rp.read()
rp.can_fetch('*',url)

True

In [3]:
session = HTMLSession()
r = session.get(url)

In [4]:
# get pagination url

findable_html = r.html
pages = findable_html.find('.list-pager', first=True).find('ul li a')
pages_url = [url]
for page in pages:
    pages_url.append('https://tiki.vn'+page.attrs['href'])

pages_url

['https://tiki.vn/dien-thoai-smartphone/c1795?_lc=Vk4wMzkwMjIwMDk%3D',
 'https://tiki.vn/dien-thoai-smartphone/c1795?_lc=Vk4wMzkwMjIwMDk%3D&page=2',
 'https://tiki.vn/dien-thoai-smartphone/c1795?_lc=Vk4wMzkwMjIwMDk%3D&page=3',
 'https://tiki.vn/dien-thoai-smartphone/c1795?_lc=Vk4wMzkwMjIwMDk%3D&page=4',
 'https://tiki.vn/dien-thoai-smartphone/c1795?_lc=Vk4wMzkwMjIwMDk%3D&page=5',
 'https://tiki.vn/dien-thoai-smartphone/c1795?_lc=Vk4wMzkwMjIwMDk%3D&page=2']

In [5]:
def create_line(column_titles):
    res = f''
    for title in column_titles:
        res += '{'+title+'}'+'\t'
        
    return res

In [9]:
cnt = 0
cnt_fail = 0
f = open('../dataset/tiki.csv', 'w', encoding='utf-8')
f.write(f'url\ttitle\tprovider\tmonitor\tos\tfront_cam\tback_cam\tcpu\tram\trom\tSD\tSIM\tbattery\tprice\n')


for i, page_url in enumerate(pages_url):
    r = session.get(page_url)
    product_list = r.html.find('.product-box-list div a')
    
    for product in product_list:
        title = product.attrs['title']
        product_url = product.attrs['href']

        r = session.get(product_url)
        price = r.html.find('#span-price', first=True).text
        provider = r.html.find('.item-brand p a', first=True).text

        specifications = r.html.find('.top-feature-item p')

        specifications_name = ['product_url','title','provider','monitor','os','front_cam', 
                               'back_cam', 'cpu', 'ram', 'rom', 'SD', 'SIM', 'battery']


        #url,title,provider,monitor,os,front_cam,back_cam,cpu,ram,rom,SD,SIM,battery=specifications[0]

        monitor = ''
        os = ''
        front_cam = ''
        back_cam = ''
        cpu = ''
        ram = ''
        rom = ''
        SD = ''
        SIM = ''
        battery = ''
        for specification in specifications:
            _text = specification.text.lower()
            if _text.find('màn hình') != -1:
                monitor += specification.text + ' '
                
            if _text.find('điều hành') != -1:
                os += specification.text + ' '
                
            if _text.find('camera trước') != -1:
                front_cam += specification.text + ' '
                
            if _text.find('camera sau') != -1:
                back_cam += specification.text + ' '
                
            if _text.find('chip') != -1 or _text.find('cpu') != -1:
                cpu += specification.text + ' '
                
            if _text.find('ram') != -1:
                ram += specification.text + ' '
                
            if _text.find('bộ nhớ') != -1:
                rom += specification.text + ' '
                
            if _text.find('sd') != -1:
                SD += specification.text + ' '
            
            if _text.find('sim') != -1:
                SIM += specification.text + ' '
                
            if _text.find('pin') != -1 or _text.find('mah') != -1:
                battery += specification.text + ' '

        
        f.write(f'{product_url}\t{title}\t{provider}\t{monitor}\t{os}\t{front_cam}\t{back_cam}\t{cpu}\t{ram}\t{rom}\t{SD}\t{SIM}\t{battery}\t{price}\n')
        cnt+=1 
        
    
    print('after page {}: total {} items'.format(i, cnt))
    

f.close()

after page 0: total 48 items
after page 1: total 96 items
after page 2: total 144 items
after page 3: total 192 items
after page 4: total 240 items
after page 5: total 288 items
